We use the `all-MiniLM-L6-v2` dense embedding model (also supported in FastEmbed) as a first-stage retriever and then refine results with `Jina Reranker v2`.

In [1]:
from fastembed import TextEmbedding
from fastembed.rerank.cross_encoder import TextCrossEncoder

**Cross encoder** là một kiến trúc mô hình học sâu được sử dụng phổ biến trong các bài toán truy xuất thông tin và xếp hạng. Dưới đây là những điểm chính về cross encoder:

- **Xử lý chung cho truy vấn và tài liệu:**

    Trong cross encoder, truy vấn và tài liệu được kết hợp (nối chuỗi) thành một đầu vào duy nhất trước khi được đưa qua mô hình (thường là BERT hoặc các transformer khác). Điều này cho phép mô hình xem xét đồng thời tất cả các token của cả truy vấn và tài liệu.

- **Tương tác sớm giữa các token:**

    Vì truy vấn và tài liệu được xử lý cùng nhau, các token của truy vấn có thể tương tác trực tiếp với các token của tài liệu thông qua các cơ chế attention. Điều này giúp mô hình nắm bắt được mối quan hệ ngữ nghĩa rất chi tiết giữa truy vấn và tài liệu.

>Cross encoder là mô hình hiệu quả trong việc đánh giá mối quan hệ giữa truy vấn và tài liệu nhưng lại tốn nhiều thời gian tính toán, nên phù hợp cho việc sắp xếp lại kết quả truy xuất chứ không phải cho việc truy xuất toàn bộ trên dữ liệu lớn.

In [2]:
TextCrossEncoder.list_supported_models()

[{'model': 'Xenova/ms-marco-MiniLM-L-6-v2',
  'size_in_GB': 0.08,
  'sources': {'hf': 'Xenova/ms-marco-MiniLM-L-6-v2'},
  'model_file': 'onnx/model.onnx',
  'description': 'MiniLM-L-6-v2 model optimized for re-ranking tasks.',
  'license': 'apache-2.0'},
 {'model': 'Xenova/ms-marco-MiniLM-L-12-v2',
  'size_in_GB': 0.12,
  'sources': {'hf': 'Xenova/ms-marco-MiniLM-L-12-v2'},
  'model_file': 'onnx/model.onnx',
  'description': 'MiniLM-L-12-v2 model optimized for re-ranking tasks.',
  'license': 'apache-2.0'},
 {'model': 'BAAI/bge-reranker-base',
  'size_in_GB': 1.04,
  'sources': {'hf': 'BAAI/bge-reranker-base'},
  'model_file': 'onnx/model.onnx',
  'description': 'BGE reranker base model for cross-encoder re-ranking.',
  'license': 'mit'},
 {'model': 'jinaai/jina-reranker-v1-tiny-en',
  'size_in_GB': 0.13,
  'sources': {'hf': 'jinaai/jina-reranker-v1-tiny-en'},
  'model_file': 'onnx/model.onnx',
  'description': 'Designed for blazing-fast re-ranking with 8K context length and fewer para

### Load model

We will vectorize a toy movie description dataset using the all-MiniLM-L6-v2 model and save the embeddings in Qdrant for first-stage retrieval.

Then, we will use a cross-encoder reranking model to rerank a small subset of data retrieved in the first stage.



In [4]:
dense_embedding_model = TextEmbedding("sentence-transformers/all-MiniLM-L6-v2")
reranker = TextCrossEncoder(model_name='jinaai/jina-reranker-v2-base-multilingual')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

### Data


In [5]:
# Movie descriptions dataset
descriptions = ["In 1431, Jeanne d'Arc is placed on trial on charges of heresy. The ecclesiastical jurists attempt to force Jeanne to recant her claims of holy visions.",
 "A film projectionist longs to be a detective, and puts his meagre skills to work when he is framed by a rival for stealing his girlfriend's father's pocketwatch.",
 "A group of high-end professional thieves start to feel the heat from the LAPD when they unknowingly leave a clue at their latest heist.",
 "A petty thief with an utter resemblance to a samurai warlord is hired as the lord's double. When the warlord later dies the thief is forced to take up arms in his place.",
 "A young boy named Kubo must locate a magical suit of armour worn by his late father in order to defeat a vengeful spirit from the past.",
 "A biopic detailing the 2 decades that Punjabi Sikh revolutionary Udham Singh spent planning the assassination of the man responsible for the Jallianwala Bagh massacre.",
 "When a machine that allows therapists to enter their patients' dreams is stolen, all hell breaks loose. Only a young female therapist, Paprika, can stop it.",
 "An ordinary word processor has the worst night of his life after he agrees to visit a girl in Soho whom he met that evening at a coffee shop.",
 "A story that revolves around drug abuse in the affluent north Indian State of Punjab and how the youth there have succumbed to it en-masse resulting in a socio-economic decline.",
 "A world-weary political journalist picks up the story of a woman's search for her son, who was taken away from her decades ago after she became pregnant and was forced to live in a convent.",
 "Concurrent theatrical ending of the TV series Neon Genesis Evangelion (1995).",
 "During World War II, a rebellious U.S. Army Major is assigned a dozen convicted murderers to train and lead them into a mass assassination mission of German officers.",
 "The toys are mistakenly delivered to a day-care center instead of the attic right before Andy leaves for college, and it's up to Woody to convince the other toys that they weren't abandoned and to return home.",
 "A soldier fighting aliens gets to relive the same day over and over again, the day restarting every time he dies.",
 "After two male musicians witness a mob hit, they flee the state in an all-female band disguised as women, but further complications set in.",
 "Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household.",
 "A renegade reporter trailing a young runaway heiress for a big story joins her on a bus heading from Florida to New York, and they end up stuck with each other when the bus leaves them behind at one of the stops.",
 "Story of 40-man Turkish task force who must defend a relay station.",
 "Spinal Tap, one of England's loudest bands, is chronicled by film director Marty DiBergi on what proves to be a fateful tour.",
 "Oskar, an overlooked and bullied boy, finds love and revenge through Eli, a beautiful but peculiar girl."]

In [6]:
descriptions_embeddings = list(
    dense_embedding_model.embed(descriptions)
)

### Upload the embeddings to Qdrant (local).

Qdrant Client offers a simple in-memory mode, allowing you to experiment locally with small data volumes.
Alternatively, you can use <span style="color:red">a free cluster</span> in Qdrant Cloud for experiments.



In [7]:
from qdrant_client import QdrantClient, models

qdrant_client = QdrantClient(":memory:") # Qdrant is running from RAM.

Crete a <span style="color:red">collection</span> with movie data

In [8]:
qdrant_client.create_collection(
    collection_name="movies",
    vectors_config={
        "embedding": models.VectorParams(
            size=384, #size of `all-MiniLM-L6-v2` embeddings
            distance=models.Distance.COSINE
        )
    }
)

True

Upload the embeddings

In [9]:
qdrant_client.upload_points(
    collection_name="movies",
    points=[
        models.PointStruct(
            id=idx,
            payload={
                "description": description
            },
            vector={
                "embedding": vector
            }
        )
        for idx, (description, vector) in enumerate(zip(descriptions, 
                                                        descriptions_embeddings))
    ],
)

### First-stage retrieval
Let’s see how relevant the results will be using only an `all-MiniLM-L6-v2-based` dense retriever.

In [10]:
query = '''A story about a strong historically significant female figure.'''
query_embedded = list(dense_embedding_model.query_embed(query))[0]

initial_retrieval = qdrant_client.query_points(
    collection_name="movies",
    using="embedding",
    query=query_embedded,
    with_payload=True,
    limit=10
)

description_hits = []
for i, hit in enumerate(initial_retrieval.points):
    print(f'''Result number {i+1} is \"{hit.payload["description"]}\"''')
    description_hits.append(hit.payload["description"])

Result number 1 is "A world-weary political journalist picks up the story of a woman's search for her son, who was taken away from her decades ago after she became pregnant and was forced to live in a convent."
Result number 2 is "Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household."
Result number 3 is "Oskar, an overlooked and bullied boy, finds love and revenge through Eli, a beautiful but peculiar girl."
Result number 4 is "A renegade reporter trailing a young runaway heiress for a big story joins her on a bus heading from Florida to New York, and they end up stuck with each other when the bus leaves them behind at one of the stops."
Result number 5 is "A story that revolves around drug abuse in the affluent north Indian State of Punjab and how the youth there have succumbed to it en-masse resulting in a socio-economic decline."
Result number 6 is "After two male musicians witness a mob hit, they

We can see that the description of “The Messenger: The Story of Joan of Arc”, which is the most fitting, appears 10th in the results.

Let’s try refining the order of the retrieved subset with `Jina Reranker v2`. It takes a query and a set of documents (movie descriptions) as input and calculates a relevance score based on token-level interactions between the query and each document.



In [12]:
new_scores = list(reranker.rerank(query, description_hits)) #returns scores between query and each document

ranking = [(i, score) for i, score in enumerate(new_scores)] #saving document indices
ranking.sort(key=lambda x: x[1], reverse=True) #sorting them in order of relevance defined by reranker

for i, rank in enumerate(ranking):
    print(f'''Reranked result number {i+1} is \"{description_hits[rank[0]]}\"''')

Reranked result number 1 is "In 1431, Jeanne d'Arc is placed on trial on charges of heresy. The ecclesiastical jurists attempt to force Jeanne to recant her claims of holy visions."
Reranked result number 2 is "Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household."
Reranked result number 3 is "Oskar, an overlooked and bullied boy, finds love and revenge through Eli, a beautiful but peculiar girl."
Reranked result number 4 is "A world-weary political journalist picks up the story of a woman's search for her son, who was taken away from her decades ago after she became pregnant and was forced to live in a convent."
Reranked result number 5 is "A renegade reporter trailing a young runaway heiress for a big story joins her on a bus heading from Florida to New York, and they end up stuck with each other when the bus leaves them behind at one of the stops."
Reranked result number 6 is "After two male music

Rerankers refine search results by reordering retrieved candidates through deeper semantic analysis. For efficiency, they should be applied **only to a small subset of retrieved results.**

Balance speed and accuracy in search by leveraging the power of rerankers!